Baldeep's Code


In [ ]:
import imaplib
import email
from email.header import decode_header
import requests
from requests.auth import HTTPBasicAuth

# Gmail and Jira details
GMAIL_USER = 'XYZ@gmail.com'
GMAIL_PASSWORD = 'abc def ghi jkl'  # Generate an App Password from your Google account
JIRA_URL = 'https://xyz.atlassian.net/rest/api/2/issue'
JIRA_USER = 'xyz@gmail.com'
JIRA_API_TOKEN = 'ABCDEFGIJKLMNOPQRSTUVWXYZ'

# IMAP Gmail connection
def connect_to_gmail():
    mail = imaplib.IMAP4_SSL("imap.gmail.com")
    mail.login(GMAIL_USER, GMAIL_PASSWORD)
    mail.select("inbox")
    return mail

# Function to search for emails from a specific sender
def search_emails_from_sender(mail, sender_email):
    status, messages = mail.search(None, f'(FROM "{sender_email}")')
    email_ids = messages[0].split()
    return email_ids

# Function to get email details (subject, body)
def get_email_details(mail, email_id):
    res, msg = mail.fetch(email_id, "(RFC822)")
    for response_part in msg:
        if isinstance(response_part, tuple):
            msg = email.message_from_bytes(response_part[1])
            subject, encoding = decode_header(msg["Subject"])[0]
            if isinstance(subject, bytes):
                subject = subject.decode(encoding or "utf-8")

            if msg.is_multipart():
                for part in msg.walk():
                    if part.get_content_type() == "text/plain":
                        body = part.get_payload(decode=True).decode()
                        return subject, body
            else:
                body = msg.get_payload(decode=True).decode()
                return subject, body
    return None, None

# Function to create Jira issue
def create_jira_issue(project_key, issue_type, summary, description):
    headers = {
        'Content-Type': 'application/json'
    }
    auth = HTTPBasicAuth(JIRA_USER, JIRA_API_TOKEN)

    data = {
    "fields": {
        "project": {
            "key": project_key
        },
        "summary": summary,
        "description": description,
        "issuetype": {
            "name": issue_type
        },
        "assignee": {
            "name": "baldeep.singh@XYZ.com"  # Replace with the username of the person you're assigning to
        }
    }
}

    response = requests.post(JIRA_URL, json=data, headers=headers, auth=auth)

    if response.status_code == 201:
        print("Issue created successfully.")
    else:
        print(f"Failed to create issue. Status code: {response.status_code}, Response: {response.text}")

# Main function to run the process
def main():
    sender_email = "baldeep.singh@XYZ.com"  # Email address to monitor
    project_key = "PROJECT_KEY"  # Jira project key
    issue_type = "Task"  # Jira issue type

    # Connect to Gmail
    mail = connect_to_gmail()

    # Search for emails from the specific sender
    email_ids = search_emails_from_sender(mail, sender_email)

    if email_ids:
        for email_id in email_ids:
            subject, body = get_email_details(mail, email_id)
            if subject and body:
                print(f"Creating Jira issue for email: {subject}")
                create_jira_issue(project_key, issue_type, subject, body)
    else:
        print(f"No new emails from {sender_email}")

    # Logout from Gmail
    mail.logout()

if __name__ == "__main__":
    main()
